# Automotive NER Assignment
------------------------------------

Second task is to use an open source LLM of your choice and write the prompt to extract the automotive domain entities from given dataset. Some examples of LLMs include LLaMA 2 7b, Phi-2, Mistral 7b etc. Explore zero shot and few-shot learning and other prompt engineering techniques.

## Meta AI Llama 2 Chat 7B LLM LangChain Text Summarization Named Entity Recognition Colab Tutorial

In [ ]:
!pip install -q transformers einops accelerate langchain bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.2/806.2 kB 42.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 82.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.4/252.4 kB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 18.5 MB/s eta 0:00:00


In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid (permission: read).
Your token has been saved in your conf

In [ ]:
!pip install sentencepiece

In [ ]:
from langchain import HuggingFacePipeline
from transformers import AutoTokenizer
import transformers
import torch

In [ ]:
model = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(model)

pipeline = transformers.pipeline(
    "text-generation",
    model = model,
    tokenizer = tokenizer,
    torch_dtype = torch.bfloat16,
    trust_remote_code = True,
    device_map = 'auto',
    max_length = 1000,
    do_sample = True,
    top_k = 10,
    num_return_sequences = 1,
    eos_token_id = tokenizer.eos_token_id
)

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [ ]:
from langchain import LLMChain, PromptTemplate

In [ ]:
llm = HuggingFacePipeline(pipeline = pipeline, model_kwargs = {'temperature' : 0})

In [ ]:
input_text = """
          conditions can result in the bottoming out the suspension and amplification of the stress
          placed on the floor truss network. the additional stress can result in the fracture of welds securing the
          floor truss network system to the chassis frame rail and/or fracture of the floor truss network support
          system. the possibility exists that there could be damage to electrical wiring and/or fuel lines which
          could potentially lead to a fire.
"""

In [ ]:
template = """
              Detect named entities in following text delimited by triple backquotes.
              We are interested in entities related to automotive domain. Labels are component, failure issue, vehicle model, corrective action only.
              Return your response in json format with spans of named entities with fields "Entity" , "Label".
              Return all the Entity and Label in the format
              [ {Entity: bottoming out the suspension, Label: Failure Issue},.....]
              ```{text}```
              Result:
          """

prompt = PromptTemplate(template=template, input_variables=["text"])

llm_chain = LLMChain (prompt=prompt, llm=llm)

In [ ]:
print(llm_chain.run(text = input_text))

ValueError: Missing some input keys: {'Entity'}